<a href="https://colab.research.google.com/github/verm024/bsd-capstone/blob/main/Capstone_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Code Steps**
<br>

1.   Import dataset from spreadsheet URL as csv 
2.   Split the data into training, validation, and test data
3.   Convert the data to input pipeline using tf.data
4.   Extract feature columns
5.   Create the architecture of the model
6.   Train, validate, and test the model
7.   Export the model to TFX format
8.   Deploy and test the deployed model

## Import Library

In [ ]:
# Import
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## Load Dataset

In [ ]:
# Load Dataset CSV
URL = 'https://docs.google.com/spreadsheets/d/1h6TldA5tUq2Vu0wHfAzHqmtSkpKHHyL6ONOaf8F4PII/export?gid=0&format=csv'
df = pd.read_csv(URL)
df

,umur,gaji,tanggungan,pekerjaan,diterima
0,45,1.5,5,8,1
1,50,2.5,6,5,1
2,60,0.0,1,0,1
3,42,1.6,3,5,1
4,63,0.2,0,1,1
...,...,...,...,...,...
57,47,2.7,1,2,0
58,58,5.7,2,10,0
59,34,2.1,3,9,0
60,19,0.2,0,4,0


## Split Train, Validation, Test

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

39 train examples
10 validation examples
13 test examples


## Convert dataframe to tf.data


In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('diterima')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
print(train_ds)

<BatchDataset shapes: ({umur: (None,), gaji: (None,), tanggungan: (None,), pekerjaan: (None,)}, (None,)), types: ({umur: tf.int64, gaji: tf.float64, tanggungan: tf.int64, pekerjaan: tf.int64}, tf.int64)>


In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['umur'])
  print('A batch of targets:', label_batch )

Every feature: ['umur', 'gaji', 'tanggungan', 'pekerjaan']
A batch of ages: tf.Tensor([20 56 66 26 38], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([0 0 1 0 0], shape=(5,), dtype=int64)


## Extract Feature Columns

In [ ]:
example_batch = next(iter(train_ds))[0]

def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [ ]:
umur = feature_column.numeric_column("umur")
demo(umur)
umur_buckets = feature_column.bucketized_column(umur, boundaries=[25, 35, 45, 55, 65])
demo(umur_buckets)

[[51.]
 [47.]
 [29.]
 [66.]
 [41.]]
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]]


In [ ]:
# 0	Tidak Bekerja
# 1	Pensiunan
# 2	PNS
# 3	Wiraswasta
# 4	Karyawan
# 5	Petani/Nelayan/Peternak
# 6	Pegawai Pemerintah
# 7	Pekerja kasar
# 8	Buruh pabrik
# 9	Pemuka Agama
# 10	Lainnya

pekerjaan = feature_column.categorical_column_with_vocabulary_list(
      'pekerjaan', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

pekerjaan_one_hot = feature_column.indicator_column(pekerjaan)
demo(pekerjaan_one_hot)

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
feature_columns = []

# numeric cols
for header in ['umur', 'gaji', 'tanggungan', 'pekerjaan']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
umur_buckets = feature_column.bucketized_column(umur, boundaries=[25, 35, 45, 55, 65])
feature_columns.append(umur_buckets)

# categorical cols
pekerjaan = feature_column.categorical_column_with_vocabulary_list(
      'pekerjaan', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

pekerjaan_one_hot = feature_column.indicator_column(pekerjaan)
feature_columns.append(pekerjaan_one_hot)

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('Age:', list(feature_batch['umur']))
  print('A batch of ages:', feature_batch['umur'])
  print('A batch of targets:', label_batch )

Every feature: ['umur', 'gaji', 'tanggungan', 'pekerjaan']
Age: [<tf.Tensor: shape=(), dtype=int64, numpy=38>, <tf.Tensor: shape=(), dtype=int64, numpy=28>, <tf.Tensor: shape=(), dtype=int64, numpy=66>, <tf.Tensor: shape=(), dtype=int64, numpy=55>, <tf.Tensor: shape=(), dtype=int64, numpy=34>, <tf.Tensor: shape=(), dtype=int64, numpy=36>, <tf.Tensor: shape=(), dtype=int64, numpy=33>, <tf.Tensor: shape=(), dtype=int64, numpy=45>, <tf.Tensor: shape=(), dtype=int64, numpy=63>, <tf.Tensor: shape=(), dtype=int64, numpy=57>, <tf.Tensor: shape=(), dtype=int64, numpy=60>, <tf.Tensor: shape=(), dtype=int64, numpy=20>, <tf.Tensor: shape=(), dtype=int64, numpy=36>, <tf.Tensor: shape=(), dtype=int64, numpy=65>, <tf.Tensor: shape=(), dtype=int64, numpy=26>, <tf.Tensor: shape=(), dtype=int64, numpy=26>, <tf.Tensor: shape=(), dtype=int64, numpy=63>, <tf.Tensor: shape=(), dtype=int64, numpy=65>, <tf.Tensor: shape=(), dtype=int64, numpy=42>, <tf.Tensor: shape=(), dtype=int64, numpy=60>, <tf.Tensor: sha

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1/2 [==============>...............] - ETA: 0s - loss: 0.7444 - accuracy: 0.5000WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'umur': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=int64>, 'gaji': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'tanggungan': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=int64>, 'pekerjaan': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=int64>}
Consider rewriting this model with the Functional API.
2/2 [==============================] - 1s 224ms/step - loss: 0.7536 - accuracy: 0.4872 - val_loss: 0.6718 - val_accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 19ms/step - loss: 0.6539 - accuracy: 0.5641 - val_loss: 0.8663 - val_accuracy: 0.6000
Epoch 3/100
2/2 [==============================] - 0s 17ms/step - loss:

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Test Accuracy", accuracy)

1/1 [==============================] - 0s 15ms/step - loss: 0.3174 - accuracy: 0.8462
Accuracy 0.8461538553237915


## Tensorflow Serving



In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  81750      0 --:--:-- --:--:-- --:--:-- 81750
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 Packages [340 B]
Hit

## Save Model

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tempfile

MODEL_DIR = tempfile.gettempdir() # MODEL_DIR='/tmp'
# MODEL_DIR = "/content/drive/MyDrive/Capstone/Model"
version = 1

export_path = os.path.join(MODEL_DIR, str(version))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

model.save(export_path, save_format="tf")

print('\nexport_path = {}'.format(export_path))
!ls -l {export_path}


Already saved a model, cleaning up

rm: cannot remove '/content/drive/MyDrive/Capstone/Model/1/variables/variables.data-00000-of-00001': Operation not permitted
rm: cannot remove '/content/drive/MyDrive/Capstone/Model/1/variables/variables.index': Operation not permitted
rm: cannot remove '/content/drive/MyDrive/Capstone/Model/1/assets': Operation not permitted
rm: cannot remove '/content/drive/MyDrive/Capstone/Model/1/keras_metadata.pb': Operation not permitted
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


PermissionDeniedError: ignored

In [ ]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['gaji'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1, 1)
        name: serving_default_gaji:0
    inputs['pekerjaan'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_pekerjaan:0
    inputs['tanggungan'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_tanggungan:0
    inputs['umur'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_umur:0
  The given S

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR
print(MODEL_DIR)

/tmp


## Serve model

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=bsd \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 3 in a separate thread.


In [ ]:
!tail server.log

2021-05-27 11:50:58.538642: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /tmp/1
2021-05-27 11:50:58.549654: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 76786 microseconds.
2021-05-27 11:50:58.551312: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-05-27 11:50:58.551471: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: bsd version: 1}
2021-05-27 11:50:58.552157: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-05-27 11:50:58.552227: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-05-27 11:50:58.552843: I tensorflow_serving/model_servers/server.cc:393] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] g

## Sample request

In [ ]:
import json
import requests

xs = [{"gaji": [test.iloc[0][1]], "pekerjaan": [int(test.iloc[0][3])], "tanggungan": [int(test.iloc[0][2])], "umur": [int(test.iloc[0][0])]}, {"gaji": [test.iloc[1][1]], "pekerjaan": [int(test.iloc[1][3])], "tanggungan": [int(test.iloc[1][2])], "umur": [int(test.iloc[1][0])]}]
print("Data:", xs)

data = json.dumps({"signature_name": "serving_default", "instances": xs})

headers = {"content-type": "application/json"}
json_response = requests.post('http://34.101.148.92:8501/v1/models/bsd:predict', data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']
print(predictions)

Data: [{'gaji': [2.2], 'pekerjaan': [5], 'tanggungan': [4], 'umur': [57]}, {'gaji': [0.0], 'pekerjaan': [0], 'tanggungan': [1], 'umur': [65]}]
[[0.833278537], [0.96526444]]
